In [30]:
from osm_integration import OSMManager
from models.Adult import Adult
from datetime import datetime
import folium

In [31]:
# Define the center point and radius for OSMManager
center_point = (48.8588897, 2.3200410217200766)  # Example: Ho Chi Minh City center
radius = 5000  # 5 km radius

# Initialize OSMManager
osm_manager = OSMManager(center_point, radius, network_type="drive")

Generating graph for center point (48.8588897, 2.3200410217200766) with radius 5000 meters...
Graph saved to cache: graph_cache/6bf208f59a960e253819f00601a6bdbd.pkl
Scanning for residential...
Scanning for residential near (48.8588897, 2.3200410217200766) within 5000 meters...
Found 97 residential.
Scanning for parks...
Scanning for parks near (48.8588897, 2.3200410217200766) within 5000 meters...
Found 782 parks.
Scanning for schools...
Scanning for schools near (48.8588897, 2.3200410217200766) within 5000 meters...
Found 1319 schools.
Scanning for workplaces...
Scanning for workplaces near (48.8588897, 2.3200410217200766) within 5000 meters...
Found 4683 workplaces.
Scanning for markets...
Scanning for markets near (48.8588897, 2.3200410217200766) within 5000 meters...
Found 714 markets.
Scanning for healthcare...
Scanning for healthcare near (48.8588897, 2.3200410217200766) within 5000 meters...
Found 1119 healthcare.
Scanning for play_areas...
Scanning for play_areas near (48.85888

In [32]:
# Create an adult person
adult = Adult(unique_id=1, type="adult", speed=11.4, osm_manager=osm_manager)

Person 1 assigned home at (48.889304564063046, 2.2933161786793215).
Person 1 assigned workplace at (48.84547102110295, 2.3568692081700244).
Person 1 assigned gym at (48.8938955, 2.2955883).
Person 1 assigned market at (48.890234, 2.291728).
Trajectory built from (48.889304564063046, 2.2933161786793215) to (48.84547102110295, 2.3568692081700244) with 110 nodes.
Trajectory built from (48.84547102110295, 2.3568692081700244) to (48.890234, 2.291728) with 96 nodes.
Trajectory built from (48.890234, 2.291728) to (48.84547102110295, 2.3568692081700244) with 105 nodes.
Trajectory built from (48.84547102110295, 2.3568692081700244) to (48.8938955, 2.2955883) with 101 nodes.
Trajectory built from (48.8938955, 2.2955883) to (48.889304564063046, 2.2933161786793215) with 10 nodes.


In [33]:
# Print the assigned waypoints
print("\n--- Assigned Waypoints ---")
for waypoint, coords in adult.waypoints.items():
    if coords:
        print(f"{waypoint.capitalize()}: {coords}")


--- Assigned Waypoints ---
Home: (48.889304564063046, 2.2933161786793215)
Workplace: (48.84547102110295, 2.3568692081700244)
Market: (48.890234, 2.291728)
Gym: (48.8938955, 2.2955883)


In [34]:
# Print the schedule
print("\n--- Schedule ---")
for movement in adult.schedule:
    print(f"{movement['start_waypoint']} → {movement['end_waypoint']} at {movement['start_time']}")


--- Schedule ---
home → workplace at 07:00:00
workplace → market at 12:00:00
market → workplace at 13:00:00
workplace → gym at 18:00:00
gym → home at 20:00:00


In [35]:
# Print the detailed schedule with trajectories
print("\n--- Detailed Schedule ---")
for movement in adult.detail_schedule:
    print(f"{movement['start_waypoint']} → {movement['end_waypoint']}")
    print(f"  Start Time: {movement['start_time']}")
    print(f"  Arrival Time: {movement['arrival_time']}")
    print(f"  Distance: {movement['distance_m']} meters")
    print(f"  Travel Time: {movement['travel_time_s']} seconds")
    print(f"  Route Nodes: {movement['route_nodes']}")


--- Detailed Schedule ---
home → workplace
  Start Time: 07:00:00
  Arrival Time: 07:11:47.648455
  Distance: 8067.192382445759 meters
  Travel Time: 707.6484546005051 seconds
  Route Nodes: [8449824562, 21292454, 5525617857, 5525617858, 27286563, 27286560, 27286551, 442309, 27235547, 5560755450, 6046934308, 303074688, 27235559, 1369651401, 367906, 4187963975, 367911, 27233732, 27234100, 297208404, 27234193, 27234192, 27234197, 31960801, 27235394, 31960431, 21661082, 981730952, 6892173207, 981730959, 32565381, 32565382, 27235305, 32565383, 32561132, 32564555, 32560746, 169326630, 32561279, 32561341, 32560725, 25934313, 368254, 25626383, 25626414, 25667136, 25934279, 25668541, 9966672196, 6230294622, 368250, 16820915, 12164999594, 368243, 4616528, 21292430, 25196652, 8395404619, 227180463, 25192858, 664130869, 160030886, 25194426, 25192856, 25192854, 470160, 25253728, 164132298, 4729508759, 470159, 25253770, 25253772, 470149, 25253722, 25253723, 367504, 24958048, 24958041, 24958038, 24

In [39]:
# Simulate position at a specific time
current_time = datetime.now().replace(hour=7, minute=10)  # Example time
position = adult.get_position_at_time(current_time)

print("\n--- Simulated Position ---")
print(f"Time: {current_time}")
print(f"Position (latitude, longitude): {position}")


--- Simulated Position ---
Time: 2025-01-12 07:10:49.844140
Position (latitude, longitude): (np.float64(48.84801545222943), np.float64(2.350739270616011))


In [40]:
# Create a Folium map centered at the person's home
home_coords = adult.waypoints["home"]
folium_map = folium.Map(location=home_coords, zoom_start=13)

# Add waypoints to the map
for waypoint, coords in adult.waypoints.items():
    if coords:
        folium.Marker(
            location=coords, 
            popup=waypoint.capitalize(), 
            icon=folium.Icon(color="blue")
        ).add_to(folium_map)

# Add trajectories to the map
for movement in adult.detail_schedule:
    route_nodes = movement["route_nodes"]
    route_coords = [(osm_manager.nodes.loc[node, "y"], osm_manager.nodes.loc[node, "x"]) for node in route_nodes]
    
    # Draw the route as a polyline
    folium.PolyLine(route_coords, color="red", weight=2.5, opacity=0.8).add_to(folium_map)

# Add current position to the map
folium.Marker(
    location=position,
    popup=f"Current Position at {current_time.time()}",
    icon=folium.Icon(color="green", icon="info-sign"),
).add_to(folium_map)

In [41]:
# Save the map to an HTML file
map_filename = "map/person_movement_map.html"
folium_map.save(map_filename)
print(f"Map saved to {map_filename}")


Map saved to map/person_movement_map.html
